In [1]:
!pip install transformers

In [14]:
!pip install unsloth

In [3]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-sprv2tgg/unsloth_22d317a27e094b91a55ba80a10d0cb5e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-sprv2tgg/unsloth_22d317a27e094b91a55ba80a10d0cb5e
  Resolved https://github.com/unslothai/unsloth.git to commit 8a055402a27c3d9643cc16947ce40311a280e69c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 7.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached unsloth_zoo-2025.4.4-py3-none-any.whl.metadata (8.0 kB)
  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  U

In [5]:
# prompt: use colab.drive to access drive data

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import zipfile


# For .zip files
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/model_chatzu.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/Colab Notebooks/")

In [9]:
prompt_style = """below is an instruction that describes a task ,paored with an input that provides further context.write a response that appropriately completes the request.before answering ,think carefully about the question and create a step-by-step cahin of thoughts to ensurea logical and accurate response.

##instruction:
you are an acadamice chatbot for zarqa university and your name is chat zu from now on you can anwser any thing about the university but always remind the user that your a zarqa chat bot when ever they ask for anything not about the university allways trie to get students and there perints to join the university try to be as helpfull, use the zarqa university to answer relevent questions,i add a algorithem to predict what courses should the student takeand to find out if the lecyurers are avalable.

##Topic:
{}

##Response:
<think>
{}
</think>
{}"""

In [10]:
from collections import deque

class Course:
    def __init__(self, name, credits, sections, category, prerequisites=[]):
        self.name = name
        self.credits = credits
        self.sections = sections  # List of possible time slots
        self.category = category  # e.g., 'core', 'elective', 'lab'
        self.prerequisites = prerequisites

def topological_sort(courses):
    in_degree = {course: 0 for course in courses}
    for course in courses:
        for prereq in course.prerequisites:
            in_degree[course] += 1

    queue = deque([course for course in courses if in_degree[course] == 0])
    sorted_courses = []

    while queue:
        course = queue.popleft()
        sorted_courses.append(course)
        for dependent in courses:
            if course in dependent.prerequisites:
                in_degree[dependent] -= 1
                if in_degree[dependent] == 0:
                    queue.append(dependent)

    return sorted_courses

def schedule_courses(courses, max_credits=18, taken_courses=set(), category_requirements={}):
    sorted_courses = topological_sort(courses)
    schedule = []
    completed_courses = set(taken_courses)
    category_credits_taken = {cat: 0 for cat in category_requirements}

    while sorted_courses:
        current_semester = []
        current_credits = 0
        occupied_time_slots = set()
        remaining_courses = []

        for course in sorted_courses:
            if course.name in completed_courses:
                continue

            if any(prereq.name not in completed_courses for prereq in course.prerequisites):
                remaining_courses.append(course)
                continue

            # Skip if category requirement is already fulfilled
            if course.category in category_requirements and category_credits_taken[course.category] >= category_requirements[course.category]:
                continue

            # Choose an available section
            chosen_section = None
            for section in course.sections:
                if section not in occupied_time_slots:
                    chosen_section = section
                    break

            if chosen_section and current_credits + course.credits <= max_credits:
                current_semester.append((course, chosen_section))
                current_credits += course.credits
                occupied_time_slots.add(chosen_section)
                category_credits_taken[course.category] += course.credits
            else:
                remaining_courses.append(course)

        if current_semester:
            schedule.append(current_semester)
            completed_courses.update(c.name for c, _ in current_semester)

        sorted_courses = remaining_courses

    return schedule

In [11]:
# Example Usage

def make_schedule():
    c1 = Course("Math 101", 3, [('Mon', '10:00-12:00'), ('Wed', '14:00-16:00')], 'core')
    c2 = Course("CS 101", 4, [('Tue', '14:00-16:00'), ('Thu', '10:00-12:00')], 'core', [c1])
    c3 = Course("Physics 101", 4, [('Mon', '12:00-14:00'), ('Fri', '16:00-18:00')], 'core')
    c4 = Course("CS 102", 4, [('Wed', '10:00-12:00'), ('Thu', '14:00-16:00')], 'core', [c2])
    c5 = Course("Elective 1", 3, [('Thu', '16:00-18:00'), ('Fri', '14:00-16:00')], 'elective')
    c6 = Course("Elective 2", 4, [('Fri', '10:00-12:00'), ('Mon', '08:00-10:00')], 'elective')
    c7 = Course("Elective 3", 3, [('Tue', '16:00-18:00'), ('Wed', '08:00-10:00')], 'elective')

    courses = [c1, c2, c3, c4, c5, c6, c7]
    taken_courses = {"Math 101"}
    category_requirements = {'core': 12, 'elective': 6}  # Example: 12 credits core, 6 credits elective

    schedule = schedule_courses(courses, taken_courses=taken_courses, category_requirements=category_requirements)

    for i, semester in enumerate(schedule):
        print(f"Semester {i+1}:")
        total_credits = sum(course.credits for course, _ in semester)
        print(f"  Total Credits: {total_credits}")
        for course, section in semester:
            print(f"  {course.name} ({course.credits} credits, {course.category}) - {section}")
make_schedule()

Semester 1:
  Total Credits: 15
  Physics 101 (4 credits, core) - ('Mon', '12:00-14:00')
  Elective 1 (3 credits, elective) - ('Thu', '16:00-18:00')
  Elective 2 (4 credits, elective) - ('Fri', '10:00-12:00')
  CS 101 (4 credits, core) - ('Tue', '14:00-16:00')
Semester 2:
  Total Credits: 4
  CS 102 (4 credits, core) - ('Wed', '10:00-12:00')


In [17]:
!pip install --upgrade fsspec>=2024.2.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.0 which is incompatible.


In [18]:
import unsloth
from unsloth import FastLanguageModel


model_path = "/content/drive/MyDrive/Colab Notebooks/content/model_chatzu"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

# Set to evaluation mode
model.eval()

AttributeError: module 'fsspec' has no attribute 'callbacks'

In [ ]:
input = "what is the zarqa university"
FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(input,"","")],return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    use_cache=True,
)
Response = tokenizer.batch_decode(outputs)
print(Response[0].split("##Response:")[1])

In [ ]:
import re

# --- Function Mapping ---
def execute_function_call(model_output):
    # The input model_output is already a string, no need to join
    func_name_match = re.search(r"<function>([^<]+)</function>", model_output)

    if not func_name_match:
        # If the pattern is not found, print the output and indicate no function was detected
        print("No function call detected in model output.")
        print(model_output)
        return model_output

    func_name = func_name_match.group(1).strip()
    print(f"Detected function call: {func_name}")

    # Call the function (no args)
    if func_name == 'make_schedule':
        result = make_schedule()
        # Convert the result to a string if necessary for printing
        return f"{func_name} result: {result}"
    else:
        return f"Error: Function '{func_name}' not found."

# --- Test the Model ---
model_output = Response[0].split("##Response:")[1]

# Execute the function
final_output = execute_function_call(model_output)
print(final_output)

Detected function call: make_schedule
Semester 1:
  Total Credits: 15
  Physics 101 (4 credits, core) - ('Mon', '12:00-14:00')
  Elective 1 (3 credits, elective) - ('Thu', '16:00-18:00')
  Elective 2 (4 credits, elective) - ('Fri', '10:00-12:00')
  CS 101 (4 credits, core) - ('Tue', '14:00-16:00')
Semester 2:
  Total Credits: 4
  CS 102 (4 credits, core) - ('Wed', '10:00-12:00')
make_schedule result: None


In [ ]:
import re
from datetime import datetime

def execute_function_call(model_output):
    # Extract day and time ranges
    day_time_ranges = re.findall(r'([A-Za-z]+)\s(\d{1,2}:\d{2}-\d{1,2}:\d{2})', model_output)

    if not day_time_ranges:
        print(model_output)  # No day and time ranges detected

    current_day = datetime.now().strftime("%A")
    current_time = datetime.now().time()

    print("Extracted Day & Time Ranges:", day_time_ranges)
    print("Current Day:", current_day)
    print("Current Time:", current_time)

    # Function to check if current time is within a given time range
    def is_time_in_range(time_range, current_time):
        start_str, end_str = time_range.split('-')
        start = datetime.strptime(start_str, "%H:%M").time()
        end = datetime.strptime(end_str, "%H:%M").time()
        return start <= current_time <= end

    # Check each extracted day and time range
    for day, time_range in day_time_ranges:
        if day.lower() == current_day.lower():
            print(f"\nToday is {current_day}, checking time range: {time_range}")
            if is_time_in_range(time_range, current_time):
                print("✅ lecturer is available")
            else:
                print("❌ lecturer is unavailable at the moment")
        else:
            print(f"\nToday is {current_day}, skipping {day}'s range: {time_range}")
model_output = Response[0].split("##Response:")[1]

final_output = execute_function_call(model_output)
print(final_output)


<think>

</think>
<function>make_schedule</function>

The function `make_schedule` generates a weekly schedule based on the course schedule provided. It takes into account the number of hours per day, the start and end days of the week, the number of weeks, and the number of days per week. The schedule is represented as a dictionary where each day of the week maps to a list of courses.

**Example:**
```python
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def make_schedule(courses, hours_per_day=5, start_day='Monday', end_day='Friday', weeks=16):
    """
    Generates a weekly schedule based on the given courses and parameters.

    Args:
        courses (list): List of courses, each represented by a dictionary.
        hours_per_day (int): Number of hours per day (default: 5).
        start_day (str): Starting day of the week (default: 'Monday').
        end_day (str): Ending day of the week (default: 'Friday').
        weeks (int): Number 